In [ ]:
import cv2 as cv 
import os 
import numpy as np 
from mtcnn import MTCNN 
from keras_facenet import FaceNet 
from sklearn.preprocessing import LabelEncoder 
from sklearn.svm import SVC 
 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
 
class FACELOADING: 
    def __init__(self, directory): 
        self.directory = directory 
        self.target_size = (160, 160) 
        self.X = [] 
        self.Y = [] 
        self.detector = MTCNN() 
        self.embedder = FaceNet() 
        self.encoder = LabelEncoder() 
        self.model = SVC(kernel='linear', probability=True) 
 
    def extract_face(self, filename): 
        img = cv.imread(filename) 
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB) 
        x, y, w, h = self.detector.detect_faces(img)[0]['box'] 
        x, y = abs(x), abs(y) 
        face = img[y:y+h, x:x+w] 
        face_arr = cv.resize(face, self.target_size) 
        return face_arr 
 
    def load_faces(self, dir): 
        FACES = [] 
        for im_name in os.listdir(dir): 
            try: 
                path = os.path.join(dir, im_name) 
                single_face = self.extract_face(path) 
                FACES.append(single_face) 
            except Exception as e: 
                pass 
        return FACES 
 
    def load_classes(self): 
        for sub_dir in os.listdir(self.directory): 
            path = os.path.join(self.directory, sub_dir) 
            FACES = self.load_faces(path) 
            labels = [sub_dir for _ in range(len(FACES))] 
            print(f"Loaded successfully: {len(labels)}") 
            self.X.extend(FACES) 
            self.Y.extend(labels) 
 
        self.encoder.fit(self.Y) 
        self.Y_encoded = self.encoder.transform(self.Y) 
        self.X_embedded = self.embedder.embeddings(np.asarray(self.X)) 
        self.model.fit(self.X_embedded, self.Y_encoded) 
 
    def recognize_face(self): 
 
 
        cap = cv.VideoCapture(0) 
        while True: 
            ret, frame = cap.read() 
            if ret: 
                frame = cv.flip(frame, 1)   
                frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB) 
                results = self.detector.detect_faces(frame) 
                if results: 
                    for result in results: 
                        x, y, w, h = result['box'] 
                        x, y = abs(x), abs(y) 
                        face = frame[y:y+h, x:x+w] 
                        face_arr = cv.resize(face, self.target_size) 
                        face_embedded = self.embedder.embeddings(np.asarray([face_arr])) 
                        pred_encoded = self.model.predict(face_embedded) 
                        pred_proba = self.model.predict_proba(face_embedded) 
                        pred_proba = np.max(pred_proba) * 100 
                        if pred_encoded in self.Y_encoded: 
                            pred_name = self.encoder.inverse_transform(pred_encoded) 
                            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2) 
                            cv.putText(frame, f"{pred_name[0]} ({pred_proba:.2f}%)", (x, y-10), 
cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2) 
                        else: 
                            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2) 
                            cv.putText(frame, "Unrecognized face: Entry denied", (x, y-10), 
cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2) 
                cv.imshow('frame', frame) 
                if cv.waitKey(1) == ord('q'): 
                    break 
            else: 
                break 
        cap.release() 
        cv.destroyAllWindows() 
 
# Define the directory containing the face dataset 
directory = r"C:\Users\ruthw\face_recognition\dataset" 
faceloading = FACELOADING(directory) 
faceloading.load_classes() 
faceloading.recognize_face()